In [48]:
from typing import Set, Dict, Tuple, List
import pandas as pd
from collections import defaultdict

# Fantasy Football MDP
We define a finite horizon Markov Decision Process (MDP) for fantasy football, where:
- states represent the list of positions drafted
- actions represent selecting the best available player at a particular position
  - we set limits on the number of players per position to draft (this prevents spamming QB, a pretty bad strategy!)
  - we define "available players" as any player whose rank is at least as great as the given pick number, and has not already been drafted
    - due to the deterministic nature of obtaining the best player at a given position at a given pick, we do not need to store the player explicitly, cutting space.
- rewards represent the total number of projected points for that player
- discount factor of 1
- actions are deterministic, so probabilities are always 1

Upper bound of 9! total states before accounting for duplicates in positions

Similarly, because our actions only consider the single best available player at each position, there's a smaller action space (no more than 6 actions per state!).

This means the MDP is tractable, and we can use policy iteration to find an optimal policy, i.e. a way to approach each round in the draft, position-by-position,
to maximize total projected team points.

In [20]:
player_points = pd.read_csv('../data/points_yahoo_pff_tiered.csv')

In [21]:
player_points.head()

,Unnamed: 0.1,Unnamed: 0,PLAYER NAME,TEAM,POS,BYE WEEK,POINTS,POSITIONAL TIER
0,0,0,Ja'Marr Chase,CIN,WR,10,320.0,1
1,1,2,Bijan Robinson,ATL,RB,5,298.0,1
2,25,25,Josh Allen,BUF,QB,7,349.0,2
3,2,1,Saquon Barkley,PHI,RB,9,296.0,1
4,3,4,Jahmyr Gibbs,DET,RB,8,296.0,1


In [22]:
# define all positions and their limits
POS_LIMITS = {'QB': 1, 'RB': 3, 'WR': 3, 'TE':1, 'FLEX':0, 'K':1, 'DST':1}
POS = list(POS_LIMITS.keys())
ROUNDS = sum(POS_LIMITS.values())

# define flex positions
FLEX_POS = {'RB', 'WR', 'TE'} # set of all FLEX positions

total_skill_spots = sum(POS_LIMITS[pos_name] for pos_name in FLEX_POS | {'FLEX'})

In [23]:
# produce a numerical encoding for each position, where:
# QB: 1
# RB: 2
# WR: 3
# TE: 4
# K: 5
# DST: 6
POS_TO_NUM = {'QB':1,
                 'RB':2,
                 'WR':3,
                 'TE':4,
                 'K':5,
                 'DST':6}

# and the corresponding decoding
NUM_TO_POS = {num: pos for pos, num in POS_TO_NUM.items()}

In [24]:
# We define a state as a list of players drafted so far, for example:
# - []
# - ['RB']
# - ['RB', 'RB', 'WR']

# Let's make a simple hash function for this, that takes in a state list and produces a corresponding number,
# which is simply the concatenation of each positional encoding, in order.
# For example: ['RB', 'WR', 'TE', 'QB'] -> 2341
# Note we need to hash because we need a way to represent the state itself as its own key (for policy or value mappings), and dicts can't be keys!
def hash(state_list: list) -> int:
    num = 0
    for pos in state_list:
        num = num * 10 + POS_TO_NUM[pos]
        
    return num

In [25]:
# Given the hash, produce its corresponding state list
# This assumes 2 states have the same hash iff they are the same state.
# This is useful for interpretability.
def unhash(hashed: int) -> List[str]:
    if hashed == 0: # special case for 0 - corresponds to []
        return []
    
    state_list = []
    nums = len(str(hashed))
    for i in range(nums):
        divisor = 10**(nums - i - 1)
        state_list.append(NUM_TO_POS[hashed // divisor])
        hashed = hashed % divisor
        
    return state_list
    

In [26]:
# determine if a FLEX player can be drafted for the given state list.
def is_flex_available(state_list: list) -> bool:
    return sum(max(state_list.count(pos_name) - POS_LIMITS[pos_name], 0) for pos_name in FLEX_POS) < POS_LIMITS['FLEX']

In [43]:
# Get every single possible state in the MDP, as an iterable set of their hashed versions.
# This is useful for policy iteration.
def get_all_states() -> Set[int]:
    states = set()

    def recurse(state_list: list) -> None:
        
        states.add(hash(state_list))
        
        if len(state_list) == ROUNDS: # Stop if it's a complete roster
            return
        
        # iterate over every single position that can be selected here
        flex_available = is_flex_available(state_list)

        for pos_name in POS_TO_NUM:
            count_at_pos = state_list.count(pos_name)

            if count_at_pos < POS_LIMITS[pos_name]:
                recurse(state_list + [pos_name])

            elif pos_name in FLEX_POS and flex_available:
                # Only use FLEX when the original position is saturated
                recurse(state_list + [pos_name])
                    
    recurse([])
    return states
            

In [45]:
print(f"Total number of states: {len(get_all_states())}")

Total number of states: 294397


In [29]:
# For a given state list, get all the available actions from it. In other words, the set of positions that can still be drafted (as strings, e.g.
# {'QB', 'RB'}), not violating the limits.
# We adhere to the idea of FLEX players, such that even if all the roster spots for a given skill position (RB, WR, TE) are taken up
# but the FLEX is still open, that position can still be drafted.
# Because FLEX is not an actual position, it will never be returned as an action.
def get_actions(state_list: list) -> Set[str]:
    actions = set()
    pos_counts = {pos_name: state_list.count(pos_name) for pos_name in POS_TO_NUM}
    flex_available = is_flex_available(state_list)
    for pos in pos_counts:
        if pos_counts[pos] >= POS_LIMITS[pos]:
            # if the position is a FLEX, and there FLEXes are not saturated, you could still draft them
            if pos in FLEX_POS and flex_available:
                actions.add(pos)
                    
        else: # otherwise, it's below the limit so we can simply draft them
            actions.add(pos)
                
    return actions

In [30]:
# For the given state list, action (pos string), pick in the draft, and set of drafted players at that position, gets the next state, reward, and player drafted, as a (next_state, float, string) tuple
# We assume the action is valid.
def get_result(state_list: list, action: str, pick: int, drafted: Set[str]) -> Tuple[List[str], float, str]:
    next_state = state_list + [action]
    #print(action)
        
    filtered = player_points[(player_points['POS'] == action) & (player_points.index >= pick - 1) & (~player_points['PLAYER NAME'].isin(drafted))]
    #print(filtered['POINTS'].idxmax())
    max_idx = filtered['POINTS'].idxmax()
    #print(max_idx)
    row = player_points.loc[max_idx]
    #print(row)
    return next_state, float(row['POINTS']), row['PLAYER NAME']

In [31]:
# calculate the current round in the draft, given the state list
# this is just the number of players they drafted, plus one
def get_round(state_list: int) -> int:
    return len(state_list) + 1

In [32]:
# calculate the current pick, given the round, first round pick number, and number of teams
# assumes snaking order
# round is one-indexed
def get_pick(round: int, first_pick_num: int, num_teams: int) -> int:
    if round % 2 != 0: # odd (first, third, fifth, ... rounds)
        return first_pick_num + (round-1) * num_teams
    else: # even (second, fourth, sixth, ... rounds)
        return (num_teams - first_pick_num + 1) + (round-1) * num_teams

In [ ]:
# Runs policy iteration for the given first pick and number of teams, returning the optimal policy as a mapping from the hashed state to the optimal action
def policy_iteration(first_pick_num: int, num_teams: int) -> Dict[int, str]:
    states = get_all_states() # hashed versions
    vals = defaultdict(int) # Value for each state
    policy = defaultdict(str) # action at each state       
    players = defaultdict(set)
    
    print("Precomputing taken player sets...")
    for state in states:
        state_list = unhash(state)
        #print(state_list)
        actions = get_actions(state_list)
        #print(actions)
        #print(actions)
        for action in actions:
            round = get_round(state_list)
            pick = get_pick(round, first_pick_num, num_teams)
                
            # calculate which players have been drafted
            # this needs to be done live, because when calculating previous picks, those picks themselves must be conditioned on their previous picks
            # to prevent duplicates.
            indices = [i for i, pos in enumerate(state_list) if pos == action]
                
            drafted = set()
            for round_zero in indices: # rounds here are zero indexed
                #print(drafted)
                pick = get_pick(round_zero+1, first_pick_num, num_teams) # add 1 for one indexing
                _, _, player = get_result([], action, pick, drafted) # just pass in empty list, we assume position is under limit
                drafted.add(player)
                
            players[(state,action)] = drafted
    
    print("Running policy iteration...")
    # Run the algorithm till policy converges!
    t = 1
    while True:
        policy_changed = False
        for state in states:
            #print(state)
            state_list = unhash(state)
            
            actions = get_actions(state_list)
            best_action = ""
            best_return = -1
            for action in actions:
                round = get_round(state_list)
                pick = get_pick(round, first_pick_num, num_teams)

                drafted = players[(state,action)]
                #print(drafted)
                #print(action)
                next_state, reward, _ = get_result(state_list, action, pick, drafted)
                
                # sum of current and future rewards
                cumulative = reward + 1*vals[hash(next_state)] # gamma = 1
                if cumulative > best_return:
                    best_action = action
                    best_return = cumulative
        
            if policy[state] != best_action:
                policy_changed = True
              
            vals[state] = best_return
            policy[state] = best_action
        
        if not policy_changed: # break once the policy does not change!
            break
        
        t += 1
    
    print(f"Finished running policy iteration after {t} iterations!")
    return policy

In [47]:
first_pick_num = 12
num_teams = 12

policy = policy_iteration(first_pick_num=first_pick_num, num_teams=num_teams)

Precomputing taken player sets...
Running policy iteration...
Finished running policy iteration after 10 iterations!


In [52]:
# Now, get the best sequence of actions!

# given a policy, produces the actions from start to end, their rewards, and the drafted player names
def get_policy_results(policy: Dict[int, str]) -> Tuple[List[str], List[float], List[str]]:

    actions = []
    rewards = []
    names = []
    starting_state_list = []
    state = hash(starting_state_list)
    for round in range(1, ROUNDS + 1):
        #print(state)
        #print(names)
        state_list = unhash(state)
    
        action = policy[state]
        pick = get_pick(round, first_pick_num, num_teams)
    
        drafted = set(names[i] for i, pos in enumerate(actions) if pos == action)
    
        next_state, reward, player_name = get_result(state_list, action, pick, drafted)
        actions.append(action)
        rewards.append(reward)
        names.append(player_name)
        state = hash(next_state)
        
    return actions, rewards, names

In [54]:
actions, rewards, names = get_policy_results(policy)
print(f"Actions: {actions}")
print(f"Rewards: {rewards}")
print(f"Total rewards: {sum(rewards)}")
print(f"Player names: {names}")

Actions: ['WR', 'RB', 'RB', 'WR', 'RB', 'TE', 'QB', 'K', 'DST', 'WR']
Rewards: [269.0, 267.0, 225.0, 235.0, 205.8, 182.0, 290.0, 164.0, 141.0, 191.0]
Total rewards: 2169.8
Player names: ['Brian Thomas', 'Bucky Irving', 'James Cook', 'Terry McLaurin', 'Aaron Jones', 'Sam LaPorta', 'Jared Goff', 'Brandon Aubrey', 'Houston Texans', 'Deebo Samuel']
